In [ ]:
!pip install scikit-learn==0.23.1 --force-reinstall 
!pip install imbalanced-learn

In [ ]:
import matplotlib.pyplot as plt
from azureml.core import Dataset
from azureml.data.dataset_factory import DataType
from azureml.data.dataset_factory import TabularDatasetFactory
from cleandata import clean_data
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-122643")

In [ ]:
ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv", validate=True, include_path=False, infer_column_types=True, set_column_types=None, separator=',', header=True, partition_format=None, support_multi_line=False, empty_as_string=False)
x, y = clean_data(ds)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42);

In [ ]:
data_train = x
data_train['y'] = y
feature_names = list(data_train.columns)

In [ ]:
plt.bar(['No', 'Yes'], data_train.y.value_counts().values, facecolor = 'lawngreen', edgecolor='lightseagreen', linewidth=0.5)
plt.title('Has the client subscribed a term deposit?', fontsize=14)
plt.xlabel('Answer')
plt.ylabel('No.')
plt.show()

In [ ]:
smote = SMOTE(random_state=0)
x_balanced, y_balanced = smote.fit_resample(x_train, y_train)

plt.bar(['No', 'Yes'], [sum(y_balanced), len(y_balanced)-sum(y_balanced)], facecolor = 'lawngreen', edgecolor='lightseagreen', linewidth=0.5)
plt.title('Has the client subscribed a term deposit? (Balanaced)', fontsize=14)
plt.xlabel('Answer')
plt.ylabel('No.')
plt.show()

In [ ]:
data_balanced = x_balanced
data_balanced['y'] = y_balanced
data_balanced.head()

In [ ]:
data_validation = x_test
data_validation['y'] = y_test
data_validation.head()

In [ ]:
plt.bar(['No', 'Yes'], data_validation.y.value_counts().values, facecolor = 'lawngreen', edgecolor='lightseagreen', linewidth=0.5)
plt.title('Has the client subscribed a term deposit? (Validation)', fontsize=14)
plt.xlabel('Answer')
plt.ylabel('No.')
plt.show()

In [ ]:
if "data" not in os.listdir():
    os.mkdir("./data")

local_path = './data/data_balanced.csv'
data_train.to_csv(local_path)

validation_path = './data/data_validation.csv'
data_validation.to_csv(validation_path)

datastore = ws.get_default_datastore()

datastore.upload(src_dir='data', target_path='data', overwrite=True)